<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/NN_1_10_Relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from pgmpy.estimators import HillClimbSearch, BicScore, AICScore, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score
from scipy.stats import entropy
import os
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------------------------------------

# Bayesian Network Data Generation 500, 1000, 1500, ..., 10000 Samples (dense)

In [43]:
#np.random.seed(1)

# Define the mappings for IR, EI, SP
ir_map = {0: 'low', 1: 'medium', 2: 'high'}
ei_map = {0: 'poor', 1: 'average', 2: 'good'}
sp_map = {0: 'decrease', 1: 'stable', 2: 'increase'}

# Define the dense Bayesian Network
dense_model = BayesianNetwork([('IR', 'EI'), ('EI', 'SP'), ('IR', 'SP')])

# Function to generate CPDs
def generate_cpds():
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    sp_probs_reshaped = sp_probs.reshape(3, -1)

    return ir_probs, ei_given_ir_probs, sp_probs_reshaped

# Save probabilities in a single CSV file
def save_probabilities(ir_probs, ei_probs, sp_probs, filename):
    # Create a DataFrame for IR probabilities
    ir_df = pd.DataFrame({
        'IR_State': ['low', 'medium', 'high'],
        'IR_Prob': ir_probs
    })

    # Create a DataFrame for EI given IR probabilities
    ei_df = pd.DataFrame(ei_probs, columns=['EI_given_IR_low', 'EI_given_IR_medium', 'EI_given_IR_high'])
    ei_df['EI_State'] = ['poor', 'average', 'good']

    # Create a DataFrame for SP given IR and EI probabilities
    sp_df = pd.DataFrame(sp_probs, columns=[
        'SP_given_IR_low_EI_poor', 'SP_given_IR_low_EI_average', 'SP_given_IR_low_EI_good',
        'SP_given_IR_medium_EI_poor', 'SP_given_IR_medium_EI_average', 'SP_given_IR_medium_EI_good',
        'SP_given_IR_high_EI_poor', 'SP_given_IR_high_EI_average', 'SP_given_IR_high_EI_good'
    ])
    sp_df['SP_State'] = ['decrease', 'stable', 'increase']

    # Combine all data into a single DataFrame
    combined_df = pd.concat([ir_df, ei_df, sp_df], axis=1)

    # Save the combined DataFrame as a single CSV file
    combined_df.to_csv(filename, index=False)

# Save outcomes in a CSV file
def save_outcomes(data_dense, filename):
    data_dense['IR'] = data_dense['IR'].map(ir_map)
    data_dense['EI'] = data_dense['EI'].map(ei_map)
    data_dense['SP'] = data_dense['SP'].map(sp_map)
    data_dense.to_csv(filename, index=False)

# Generate datasets for different sample sizes for the dense model
sample_sizes = range(500, 10500, 500)
for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_given_ir_probs, sp_probs_reshaped = generate_cpds()

    # Define CPDs for the dense model
    cpd_ir = TabularCPD(variable='IR', variable_card=3, values=[[ir_probs[0]], [ir_probs[1]], [ir_probs[2]]])
    cpd_ei_dense = TabularCPD(variable='EI', variable_card=3,
                              values=ei_given_ir_probs,
                              evidence=['IR'], evidence_card=[3])
    cpd_sp_dense = TabularCPD(variable='SP', variable_card=3,
                              values=sp_probs_reshaped,
                              evidence=['IR', 'EI'], evidence_card=[3, 3])

    dense_model.add_cpds(cpd_ir, cpd_ei_dense, cpd_sp_dense)

    # Check if the model is valid
    assert dense_model.check_model()

    # Generate samples
    sampler_dense = BayesianModelSampling(dense_model)
    data_dense = sampler_dense.forward_sample(size=size)

    # Save probabilities in one file
    save_probabilities(ir_probs, ei_given_ir_probs, sp_probs_reshaped, f'probabilities_dense_{size}.csv')

    # Save outcomes (low, medium, high) in another file
    save_outcomes(data_dense, f'outcomes_dense_{size}.csv')

# Notify the user that the process is done
print("Data generation and saving complete for the dense model!")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Data generation and saving complete for the dense model!


# Bayesian Network Data Generation 500, 1000, 1500, ..., 10000 Samples (sparse)

In [44]:
#np.random.seed(187)

# Define the mappings for IR, EI, SP
ir_map = {0: 'low', 1: 'medium', 2: 'high'}
ei_map = {0: 'poor', 1: 'average', 2: 'good'}
sp_map = {0: 'decrease', 1: 'stable', 2: 'increase'}

# Define the sparse Bayesian Network
sparse_model = BayesianNetwork([('IR', 'SP'), ('EI', 'SP')])

# Function to generate CPDs for the sparse model
def generate_cpds_sparse():
    # Generate probabilities for IR (unconditional)
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate unconditional probabilities for EI (no dependency on IR)
    ei_probs = np.random.rand(3)
    ei_probs /= ei_probs.sum()

    # Generate conditional probabilities for SP given IR and EI
    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    sp_probs_reshaped = sp_probs.reshape(3, -1)

    return ir_probs, ei_probs, sp_probs_reshaped

# Save probabilities in a single CSV file
def save_probabilities_sparse(ir_probs, ei_probs, sp_probs, filename):
    # Create a DataFrame for IR probabilities
    ir_df = pd.DataFrame({
        'IR_State': ['low', 'medium', 'high'],
        'IR_Prob': ir_probs
    })

    # Create a DataFrame for EI probabilities (since it's not conditional on IR)
    ei_df = pd.DataFrame({
        'EI_State': ['poor', 'average', 'good'],
        'EI_Prob': ei_probs
    })

    # Create a DataFrame for SP given IR and EI probabilities
    sp_df = pd.DataFrame(sp_probs, columns=[
        'SP_given_IR_low_EI_poor', 'SP_given_IR_low_EI_average', 'SP_given_IR_low_EI_good',
        'SP_given_IR_medium_EI_poor', 'SP_given_IR_medium_EI_average', 'SP_given_IR_medium_EI_good',
        'SP_given_IR_high_EI_poor', 'SP_given_IR_high_EI_average', 'SP_given_IR_high_EI_good'
    ])
    sp_df['SP_State'] = ['decrease', 'stable', 'increase']

    # Combine all data into a single DataFrame
    combined_df = pd.concat([ir_df, ei_df, sp_df], axis=1)

    # Save the combined DataFrame as a single CSV file
    combined_df.to_csv(filename, index=False)

# Save outcomes in a CSV file
def save_outcomes_sparse(data_sparse, filename):
    data_sparse['IR'] = data_sparse['IR'].map(ir_map)
    data_sparse['EI'] = data_sparse['EI'].map(ei_map)
    data_sparse['SP'] = data_sparse['SP'].map(sp_map)
    data_sparse.to_csv(filename, index=False)

# Generate datasets for different sample sizes for the sparse model
sample_sizes = range(500, 10500, 500)
for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_probs, sp_probs_reshaped = generate_cpds_sparse()

    # Define CPDs for the sparse model
    cpd_ir = TabularCPD(variable='IR', variable_card=3, values=[[ir_probs[0]], [ir_probs[1]], [ir_probs[2]]])
    cpd_ei_sparse = TabularCPD(variable='EI', variable_card=3, values=[[ei_probs[0]], [ei_probs[1]], [ei_probs[2]]])
    cpd_sp_sparse = TabularCPD(variable='SP', variable_card=3,
                               values=sp_probs_reshaped,
                               evidence=['IR', 'EI'], evidence_card=[3, 3])

    sparse_model.add_cpds(cpd_ir, cpd_ei_sparse, cpd_sp_sparse)

    # Check if the model is valid
    assert sparse_model.check_model()

    # Generate samples
    sampler_sparse = BayesianModelSampling(sparse_model)
    data_sparse = sampler_sparse.forward_sample(size=size)

    # Save probabilities in one file
    save_probabilities_sparse(ir_probs, ei_probs, sp_probs_reshaped, f'probabilities_sparse_{size}.csv')

    # Save outcomes (low, medium, high) in another file
    save_outcomes_sparse(data_sparse, f'outcomes_sparse_{size}.csv')

# Notify the user that the process is done
print("Data generation and saving complete for the sparse model!")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Data generation and saving complete for the sparse model!


In [45]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dense dataset for the current sample size
    dense_data_file = f'outcomes_dense_{sample_size}.csv'
    df_dense = pd.read_csv(dense_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_dense['IR_encoded'] = df_dense['IR'].map(ir_map)
    df_dense['EI_encoded'] = df_dense['EI'].map(ei_map)
    df_dense['SP_encoded'] = df_dense['SP'].map(sp_map)

    # Define the Hill-Climb structure learning algorithm
    hc = HillClimbSearch(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']])
    scoring_method = BicScore(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']])

    # Estimate the best structure
    best_dag = hc.estimate(scoring_method=scoring_method)
    best_model = BayesianNetwork(best_dag.edges())

    # Check if all nodes are included in the learned structure
    nodes_in_structure = set(best_model.nodes())
    required_nodes = {'IR_encoded', 'EI_encoded', 'SP_encoded'}

    if not required_nodes.issubset(nodes_in_structure):
        print("\nNot all nodes are connected. Adding a dummy variable.")
        # Add a dummy variable to the dataset
        df_dense['dummy'] = 0

        # Re-estimate the structure with the dummy variable
        hc = HillClimbSearch(df_dense)
        scoring_method = BicScore(df_dense)
        best_dag = hc.estimate(scoring_method=scoring_method)
        best_model = BayesianNetwork(best_dag.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples:")
    print(best_model.edges())

    # Calculate and display the BIC score
    bic_score = scoring_method.score(best_model)
    print(f"\nBIC Score for {sample_size} samples: {bic_score:.4f}")

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model.fit(df_dense, estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

    # Create an inference object for the best model
    inference = VariableElimination(best_model)

    # Placeholder to store predictions
    predicted_sp_labels = []

    # Loop through each row in the dense dataset to make predictions
    for index, row in df_dense.iterrows():
        # Prepare the evidence from the dataset (IR_encoded and EI_encoded)
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference to predict the distribution for SP_encoded (Stock Price)
        predicted_sp_distribution = inference.query(variables=['SP_encoded'], evidence=sample_input)

        # Extract the most likely SP_encoded class
        predicted_sp_class = predicted_sp_distribution.values.argmax()
        sp_reverse_map = {0: 'decrease', 1: 'stable', 2: 'increase'}
        predicted_sp_label = sp_reverse_map[predicted_sp_class]

        # Store the predicted label
        predicted_sp_labels.append(predicted_sp_label)

    # Convert the list of predicted labels into a DataFrame for easier comparison
    predicted_results_df = pd.DataFrame({
        'IR': df_dense['IR'],  # Original IR column
        'EI': df_dense['EI'],  # Original EI column
        'Predicted_SP': predicted_sp_labels  # Predicted SP column
    })

    # Add the actual SP values for comparison
    predicted_results_df['Actual_SP'] = df_dense['SP']

    # Calculate accuracy of predictions
    accuracy = accuracy_score(predicted_results_df['Actual_SP'], predicted_results_df['Predicted_SP'])
    print(f"\nPrediction Accuracy for {sample_size} samples: {accuracy:.4f}")

    # Display the first few rows of predictions
    print(f"\nPredicted Results for Dense Data (First 10 rows) for {sample_size} samples:")
    print(predicted_results_df.head(10))

# Notify the user that the process is done
print("\nProcessing complete for all sample sizes.")


Processing sample size: 500


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 500 samples:
[('IR_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded')]

BIC Score for 500 samples: -1530.3002

CPD of IR_encoded
+---------------+-------+
| IR_encoded(0) | 0.572 |
+---------------+-------+
| IR_encoded(1) | 0.234 |
+---------------+-------+
| IR_encoded(2) | 0.194 |
+---------------+-------+

CPD of SP_encoded
+---------------+---------------------+-----+---------------+---------------------+
| EI_encoded    | EI_encoded(0)       | ... | EI_encoded(2) | EI_encoded(2)       |
+---------------+---------------------+-----+---------------+---------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(1) | IR_encoded(2)       |
+---------------+---------------------+-----+---------------+---------------------+
| SP_encoded(0) | 0.4444444444444444  | ... | 0.525         | 0.14285714285714285 |
+---------------+---------------------+-----+---------------+---------------------+
| SP_encoded(1) | 0.3939393939393939  | ... | 0.

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples:
[('EI_encoded', 'IR_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

BIC Score for 1000 samples: -2818.7869

CPD of EI_encoded
+---------------+--------------------+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)      | SP_encoded(1)       | SP_encoded(2)       |
+---------------+--------------------+---------------------+---------------------+
| EI_encoded(0) | 0.3473684210526316 | 0.2659846547314578  | 0.5061728395061729  |
+---------------+--------------------+---------------------+---------------------+
| EI_encoded(1) | 0.3894736842105263 | 0.19437340153452684 | 0.4012345679012346  |
+---------------+--------------------+---------------------+---------------------+
| EI_encoded(2) | 0.2631578947368421 | 0.5396419437340153  | 0.09259259259259259 |
+---------------+--------------------+---------------------+---------------------+

CPD of IR_encoded
+---------------+---------------------+

...


Learned Structure (Edges) for 9000 samples:
[('EI_encoded', 'IR_encoded'), ('SP_encoded', 'IR_encoded'), ('SP_encoded', 'EI_encoded')]

BIC Score for 9000 samples: -27418.3816

CPD of EI_encoded
+---------------+---------------------+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(0) | 0.39520958083832336 | 0.31519274376417233 | 0.34797738147020446 |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(1) | 0.3394264103372203  | 0.27380952380952384 | 0.48325358851674644 |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(2) | 0.2653640088244564  | 0.41099773242630383 | 0.16876903001304916 |
+---------------+---------------------+---------------------+---------------------+

CPD of IR_encoded
+---------------+------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9500 samples:
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

BIC Score for 9500 samples: -28594.8663

CPD of IR_encoded
+---------------+---------------------+---------------------+--------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)      |
+---------------+---------------------+---------------------+--------------------+
| IR_encoded(0) | 0.09686003193187866 | 0.31178608515057116 | 0.2767312284425577 |
+---------------+---------------------+---------------------+--------------------+
| IR_encoded(1) | 0.4912187333688132  | 0.3637071651090343  | 0.432740780047758  |
+---------------+---------------------+---------------------+--------------------+
| IR_encoded(2) | 0.41192123469930814 | 0.3245067497403946  | 0.2905279915096843 |
+---------------+---------------------+---------------------+--------------------+

CPD of EI_encoded
+---------------+---------------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples:
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

BIC Score for 10000 samples: -28120.0358

CPD of IR_encoded
+---------------+---------------------+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(0) | 0.6909403669724771  | 0.6958066808813077  | 0.6957660410301179  |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(1) | 0.27780963302752293 | 0.22032693674484718 | 0.24487123526844173 |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(2) | 0.03125             | 0.08386638237384506 | 0.05936272370144042 |
+---------------+---------------------+---------------------+---------------------+

CPD of EI_encoded
+---------------+----------

## AIC

In [46]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dense dataset for the current sample size
    dense_data_file = f'outcomes_dense_{sample_size}.csv'
    df_dense = pd.read_csv(dense_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_dense['IR_encoded'] = df_dense['IR'].map(ir_map)
    df_dense['EI_encoded'] = df_dense['EI'].map(ei_map)
    df_dense['SP_encoded'] = df_dense['SP'].map(sp_map)

    # Define the Hill-Climb structure learning algorithm
    hc = HillClimbSearch(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']])
    scoring_method = AICScore(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']])  # Use AICScore instead of BicScore

    # Estimate the best structure
    best_dag = hc.estimate(scoring_method=scoring_method)
    best_model = BayesianNetwork(best_dag.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples (AIC):")
    print(best_model.edges())

    # Calculate and display the AIC score
    aic_score = scoring_method.score(best_model)
    print(f"\nAIC Score for {sample_size} samples: {aic_score:.4f}")

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model.fit(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']], estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

    # Create an inference object for the best model
    inference = VariableElimination(best_model)

    # Placeholder to store predictions
    predicted_sp_labels = []

    # Loop through each row in the dense dataset to make predictions
    for index, row in df_dense.iterrows():
        # Prepare the evidence from the dataset (IR_encoded and EI_encoded)
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference to predict the distribution for SP_encoded (Stock Price)
        predicted_sp_distribution = inference.query(variables=['SP_encoded'], evidence=sample_input)

        # Extract the most likely SP_encoded class
        predicted_sp_class = predicted_sp_distribution.values.argmax()
        sp_reverse_map = {0: 'decrease', 1: 'stable', 2: 'increase'}
        predicted_sp_label = sp_reverse_map[predicted_sp_class]

        # Store the predicted label
        predicted_sp_labels.append(predicted_sp_label)

    # Convert the list of predicted labels into a DataFrame for easier comparison
    predicted_results_df = pd.DataFrame({
        'IR': df_dense['IR'],  # Original IR column
        'EI': df_dense['EI'],  # Original EI column
        'Predicted_SP': predicted_sp_labels  # Predicted SP column
    })

    # Add the actual SP values for comparison
    predicted_results_df['Actual_SP'] = df_dense['SP']

    # Calculate accuracy of predictions
    accuracy = accuracy_score(predicted_results_df['Actual_SP'], predicted_results_df['Predicted_SP'])
    print(f"\nPrediction Accuracy for {sample_size} samples (AIC): {accuracy:.4f}")

    # Display the first few rows of predictions
    print(f"\nPredicted Results for Dense Data (First 10 rows) for {sample_size} samples (AIC):")
    print(predicted_results_df.head(10))

    # Save the results if needed
    results_filename = f'predicted_results_aic_{sample_size}.csv'
    predicted_results_df.to_csv(results_filename, index=False)
    print(f"\nResults saved to {results_filename}")

# Notify the user that the process is done
print("\nProcessing complete for all sample sizes using AIC.")


Processing sample size: 500


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 500 samples (AIC):
[('IR_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded')]

AIC Score for 500 samples: -1483.9395

CPD of IR_encoded
+---------------+-------+
| IR_encoded(0) | 0.572 |
+---------------+-------+
| IR_encoded(1) | 0.234 |
+---------------+-------+
| IR_encoded(2) | 0.194 |
+---------------+-------+

CPD of SP_encoded
+---------------+---------------------+-----+---------------+---------------------+
| EI_encoded    | EI_encoded(0)       | ... | EI_encoded(2) | EI_encoded(2)       |
+---------------+---------------------+-----+---------------+---------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(1) | IR_encoded(2)       |
+---------------+---------------------+-----+---------------+---------------------+
| SP_encoded(0) | 0.4444444444444444  | ... | 0.525         | 0.14285714285714285 |
+---------------+---------------------+-----+---------------+---------------------+
| SP_encoded(1) | 0.3939393939393939  | ..

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

AIC Score for 1000 samples: -2754.9861

CPD of IR_encoded
+---------------+----------------------+---------------------+----------------------+
| SP_encoded    | SP_encoded(0)        | SP_encoded(1)       | SP_encoded(2)        |
+---------------+----------------------+---------------------+----------------------+
| IR_encoded(0) | 0.21754385964912282  | 0.43478260869565216 | 0.23765432098765432  |
+---------------+----------------------+---------------------+----------------------+
| IR_encoded(1) | 0.042105263157894736 | 0.03836317135549872 | 0.027777777777777776 |
+---------------+----------------------+---------------------+----------------------+
| IR_encoded(2) | 0.7403508771929824   | 0.5268542199488491  | 0.7345679012345679   |
+---------------+----------------------+---------------------+----------------------+

CPD of EI_encoded
+-----

...


Learned Structure (Edges) for 9000 samples (AIC):
[('EI_encoded', 'IR_encoded'), ('SP_encoded', 'IR_encoded'), ('SP_encoded', 'EI_encoded')]

AIC Score for 9000 samples: -27326.0168

CPD of EI_encoded
+---------------+---------------------+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(0) | 0.39520958083832336 | 0.31519274376417233 | 0.34797738147020446 |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(1) | 0.3394264103372203  | 0.27380952380952384 | 0.48325358851674644 |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(2) | 0.2653640088244564  | 0.41099773242630383 | 0.16876903001304916 |
+---------------+---------------------+---------------------+---------------------+

CPD of IR_encoded
+---------------+------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9500 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

AIC Score for 9500 samples: -28501.7986

CPD of IR_encoded
+---------------+---------------------+---------------------+--------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)      |
+---------------+---------------------+---------------------+--------------------+
| IR_encoded(0) | 0.09686003193187866 | 0.31178608515057116 | 0.2767312284425577 |
+---------------+---------------------+---------------------+--------------------+
| IR_encoded(1) | 0.4912187333688132  | 0.3637071651090343  | 0.432740780047758  |
+---------------+---------------------+---------------------+--------------------+
| IR_encoded(2) | 0.41192123469930814 | 0.3245067497403946  | 0.2905279915096843 |
+---------------+---------------------+---------------------+--------------------+

CPD of EI_encoded
+---------------+---------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

AIC Score for 10000 samples: -28026.3014

CPD of IR_encoded
+---------------+---------------------+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(0) | 0.6909403669724771  | 0.6958066808813077  | 0.6957660410301179  |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(1) | 0.27780963302752293 | 0.22032693674484718 | 0.24487123526844173 |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(2) | 0.03125             | 0.08386638237384506 | 0.05936272370144042 |
+---------------+---------------------+---------------------+---------------------+

CPD of EI_encoded
+---------------+----

## BIC

In [47]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size} (Sparse Data with BIC)")

    # Load the sparse dataset for the current sample size
    sparse_data_file = f'outcomes_sparse_{sample_size}.csv'
    df_sparse = pd.read_csv(sparse_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_sparse['IR_encoded'] = df_sparse['IR'].map(ir_map)
    df_sparse['EI_encoded'] = df_sparse['EI'].map(ei_map)
    df_sparse['SP_encoded'] = df_sparse['SP'].map(sp_map)

    # Define the Hill-Climb structure learning algorithm
    hc_bic = HillClimbSearch(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']])
    scoring_method_bic = BicScore(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']])

    # Estimate the best structure using BIC
    best_dag_bic = hc_bic.estimate(scoring_method=scoring_method_bic)

    # Ensure all required nodes are present in the model, even if not connected
    best_model_bic = BayesianNetwork()
    best_model_bic.add_nodes_from(['IR_encoded', 'EI_encoded', 'SP_encoded'])  # Add all nodes
    best_model_bic.add_edges_from(best_dag_bic.edges())  # Add edges from the learned structure

    # Check if all nodes are included in the learned structure
    nodes_in_structure_bic = set(best_model_bic.nodes())
    required_nodes = {'IR_encoded', 'EI_encoded', 'SP_encoded'}

    if not required_nodes.issubset(nodes_in_structure_bic):
        print("\nNot all nodes are connected. Adding a dummy variable and ensuring all required nodes are present.")
        # Add a dummy variable to the dataset
        df_sparse['Dummy_Node'] = 1  # Constant dummy node

        # Ensure all required nodes are in the model by adding edges with the dummy node
        for node in required_nodes:
            if node not in nodes_in_structure_bic:
                best_model_bic.add_edge('Dummy_Node', node)

        # Re-estimate the structure with the dummy variable
        hc_bic = HillClimbSearch(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded', 'Dummy_Node']])
        scoring_method_bic = BicScore(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded', 'Dummy_Node']])
        best_dag_bic = hc_bic.estimate(scoring_method=scoring_method_bic)
        best_model_bic = BayesianNetwork(best_dag_bic.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples (BIC):")
    print(best_model_bic.edges())

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model_bic.fit(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']], estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model_bic.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model_bic.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

    # Create an inference object for the BIC model
    inference_bic = VariableElimination(best_model_bic)

    # Placeholder to store predictions
    predicted_sp_labels_bic = []

    # Loop through each row in the dataset to make predictions using BIC
    for index, row in df_sparse.iterrows():
        # Prepare the evidence from the dataset (IR_encoded and EI_encoded)
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference to predict the distribution for SP_encoded (Stock Price)
        predicted_sp_distribution_bic = inference_bic.query(variables=['SP_encoded'], evidence=sample_input)

        # Extract the most likely SP_encoded class
        predicted_sp_class_bic = predicted_sp_distribution_bic.values.argmax()
        sp_reverse_map = {0: 'decrease', 1: 'stable', 2: 'increase'}
        predicted_sp_label_bic = sp_reverse_map[predicted_sp_class_bic]

        # Store the predicted label
        predicted_sp_labels_bic.append(predicted_sp_label_bic)

    # Convert the list of predicted labels into a DataFrame for easier comparison
    predicted_results_df_bic = pd.DataFrame({
        'IR': df_sparse['IR'],  # Original IR column
        'EI': df_sparse['EI'],  # Original EI column
        'Predicted_SP': predicted_sp_labels_bic  # Predicted SP column
    })

    # Add the actual SP values for comparison
    predicted_results_df_bic['Actual_SP'] = df_sparse['SP']

    # Calculate accuracy of predictions for BIC
    accuracy_bic = accuracy_score(predicted_results_df_bic['Actual_SP'], predicted_results_df_bic['Predicted_SP'])
    print(f"\nPrediction Accuracy for {sample_size} samples (BIC): {accuracy_bic:.4f}")

    # Display the first few rows of predictions for BIC
    print(f"\nPredicted Results for Sparse Data (First 10 rows) for {sample_size} samples (BIC):")
    print(predicted_results_df_bic.head(10))

    # Calculate the BIC score for the Bayesian Network model
    bic_score_value = scoring_method_bic.score(best_model_bic)

    # Print the BIC score
    print(f"\nBIC Score for {sample_size} samples: {bic_score_value:.4f}")

# Notify the user that the process is done
print("\nProcessing complete for all sample sizes using BIC (Sparse Data).")


Processing sample size: 500 (Sparse Data with BIC)


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 500 samples (BIC):
[('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+---------------------+---------------+----------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1) | SP_encoded(2)        |
+---------------+---------------------+---------------+----------------------+
| IR_encoded(0) | 0.6022727272727273  | 0.734375      | 0.8133802816901409   |
+---------------+---------------------+---------------+----------------------+
| IR_encoded(1) | 0.18181818181818182 | 0.0390625     | 0.017605633802816902 |
+---------------+---------------------+---------------+----------------------+
| IR_encoded(2) | 0.2159090909090909  | 0.2265625     | 0.16901408450704225  |
+---------------+---------------------+---------------+----------------------+

CPD of EI_encoded
+---------------+---------------------+---------------+----------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1) | SP_en

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples (BIC):
[('IR_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded')]

CPD of IR_encoded
+---------------+-------+
| IR_encoded(0) | 0.006 |
+---------------+-------+
| IR_encoded(1) | 0.482 |
+---------------+-------+
| IR_encoded(2) | 0.512 |
+---------------+-------+

CPD of EI_encoded
+---------------+-------+
| EI_encoded(0) | 0.404 |
+---------------+-------+
| EI_encoded(1) | 0.372 |
+---------------+-------+
| EI_encoded(2) | 0.224 |
+---------------+-------+

CPD of SP_encoded
+---------------+---------------+-----+---------------------+---------------------+
| EI_encoded    | EI_encoded(0) | ... | EI_encoded(2)       | EI_encoded(2)       |
+---------------+---------------+-----+---------------------+---------------------+
| IR_encoded    | IR_encoded(0) | ... | IR_encoded(1)       | IR_encoded(2)       |
+---------------+---------------+-----+---------------------+---------------------+
| SP_encoded(0) | 0.0           | ... | 0.85436

...


Learned Structure (Edges) for 9000 samples (BIC):
[('EI_encoded', 'IR_encoded'), ('SP_encoded', 'IR_encoded'), ('SP_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+---------------------+-----+---------------------+---------------------+
| EI_encoded    | EI_encoded(0)       | ... | EI_encoded(2)       | EI_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | ... | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(0) | 0.16732617297908423 | ... | 0.11569416498993963 | 0.082687338501292   |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(1) | 0.4652345958168457  | ... | 0.5503018108651911  | 0.31976744186046513 |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(2) | 0.367439231

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9500 samples (BIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+---------------------+----------------------+----------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)        | SP_encoded(2)        |
+---------------+---------------------+----------------------+----------------------+
| IR_encoded(0) | 0.76309963099631    | 0.9118564742589703   | 0.8970657832465688   |
+---------------+---------------------+----------------------+----------------------+
| IR_encoded(1) | 0.16605166051660517 | 0.039781591263650544 | 0.0922858495030762   |
+---------------+---------------------+----------------------+----------------------+
| IR_encoded(2) | 0.07084870848708487 | 0.0483619344773791   | 0.010648367250354946 |
+---------------+---------------------+----------------------+----------------------+

CPD of EI_encoded
+---------------+---------------------+-----+-

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples (BIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+---------------------+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(0) | 0.2545334652159578  | 0.22393822393822393 | 0.30370562321845035 |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(1) | 0.5858885591823277  | 0.46782496782496785 | 0.5172324436382483  |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(2) | 0.15957797560171447 | 0.3082368082368082  | 0.17906193314330138 |
+---------------+---------------------+---------------------+---------------------+

CPD of EI_encoded
+---------------+---------------------+-----+------------------

## AIC

In [48]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size} (Sparse Data with AIC)")

    # Load the sparse dataset for the current sample size
    sparse_data_file = f'outcomes_sparse_{sample_size}.csv'
    df_sparse = pd.read_csv(sparse_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_sparse['IR_encoded'] = df_sparse['IR'].map(ir_map)
    df_sparse['EI_encoded'] = df_sparse['EI'].map(ei_map)
    df_sparse['SP_encoded'] = df_sparse['SP'].map(sp_map)

    # Define the Hill-Climb structure learning algorithm
    hc_aic = HillClimbSearch(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']])
    scoring_method_aic = AICScore(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']])

    # Estimate the best structure using AIC
    best_dag_aic = hc_aic.estimate(scoring_method=scoring_method_aic)

    # Ensure all required nodes are present in the model, even if not connected
    best_model_aic = BayesianNetwork()
    best_model_aic.add_nodes_from(['IR_encoded', 'EI_encoded', 'SP_encoded'])  # Add all nodes
    best_model_aic.add_edges_from(best_dag_aic.edges())  # Add edges from the learned structure

    # Check if all nodes are included in the learned structure
    nodes_in_structure_aic = set(best_model_aic.nodes())
    required_nodes = {'IR_encoded', 'EI_encoded', 'SP_encoded'}

    if not required_nodes.issubset(nodes_in_structure_aic):
        print("\nNot all nodes are connected. Adding a dummy variable and ensuring all required nodes are present.")
        # Add a dummy variable to the dataset
        df_sparse['Dummy_Node'] = 1  # Constant dummy node

        # Ensure all required nodes are in the model by adding edges with the dummy node
        for node in required_nodes:
            if node not in nodes_in_structure_aic:
                best_model_aic.add_edge('Dummy_Node', node)

        # Re-estimate the structure with the dummy variable
        hc_aic = HillClimbSearch(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded', 'Dummy_Node']])
        scoring_method_aic = AICScore(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded', 'Dummy_Node']])
        best_dag_aic = hc_aic.estimate(scoring_method=scoring_method_aic)
        best_model_aic = BayesianNetwork(best_dag_aic.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples (AIC):")
    print(best_model_aic.edges())

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model_aic.fit(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']], estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model_aic.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model_aic.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

    # Create an inference object for the AIC model
    inference_aic = VariableElimination(best_model_aic)

    # Placeholder to store predictions
    predicted_sp_labels_aic = []

    # Loop through each row in the dataset to make predictions using AIC
    for index, row in df_sparse.iterrows():
        # Prepare the evidence from the dataset (IR_encoded and EI_encoded)
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference to predict the distribution for SP_encoded (Stock Price)
        predicted_sp_distribution_aic = inference_aic.query(variables=['SP_encoded'], evidence=sample_input)

        # Extract the most likely SP_encoded class
        predicted_sp_class_aic = predicted_sp_distribution_aic.values.argmax()
        sp_reverse_map = {0: 'decrease', 1: 'stable', 2: 'increase'}
        predicted_sp_label_aic = sp_reverse_map[predicted_sp_class_aic]

        # Store the predicted label
        predicted_sp_labels_aic.append(predicted_sp_label_aic)

    # Convert the list of predicted labels into a DataFrame for easier comparison
    predicted_results_df_aic = pd.DataFrame({
        'IR': df_sparse['IR'],  # Original IR column
        'EI': df_sparse['EI'],  # Original EI column
        'Predicted_SP': predicted_sp_labels_aic  # Predicted SP column
    })

    # Add the actual SP values for comparison
    predicted_results_df_aic['Actual_SP'] = df_sparse['SP']

    # Calculate accuracy of predictions for AIC
    accuracy_aic = accuracy_score(predicted_results_df_aic['Actual_SP'], predicted_results_df_aic['Predicted_SP'])
    print(f"\nPrediction Accuracy for {sample_size} samples (AIC): {accuracy_aic:.4f}")

    # Display the first few rows of predictions for AIC
    print(f"\nPredicted Results for Sparse Data (First 10 rows) for {sample_size} samples (AIC):")
    print(predicted_results_df_aic.head(10))

    # Calculate the AIC score for the Bayesian Network model
    aic_score_value = scoring_method_aic.score(best_model_aic)

    # Print the AIC score
    print(f"\nAIC Score for {sample_size} samples: {aic_score_value:.4f}")

# Notify the user that the process is done
print("\nProcessing complete for all sample sizes using AIC (Sparse Data).")


Processing sample size: 500 (Sparse Data with AIC)


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 500 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+---------------------+---------------+----------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1) | SP_encoded(2)        |
+---------------+---------------------+---------------+----------------------+
| IR_encoded(0) | 0.6022727272727273  | 0.734375      | 0.8133802816901409   |
+---------------+---------------------+---------------+----------------------+
| IR_encoded(1) | 0.18181818181818182 | 0.0390625     | 0.017605633802816902 |
+---------------+---------------------+---------------+----------------------+
| IR_encoded(2) | 0.2159090909090909  | 0.2265625     | 0.16901408450704225  |
+---------------+---------------------+---------------+----------------------+

CPD of EI_encoded
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded    | IR

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+-----------------------+---------------------+----------------------+
| SP_encoded    | SP_encoded(0)         | SP_encoded(1)       | SP_encoded(2)        |
+---------------+-----------------------+---------------------+----------------------+
| IR_encoded(0) | 0.0026455026455026454 | 0.01020408163265306 | 0.006097560975609756 |
+---------------+-----------------------+---------------------+----------------------+
| IR_encoded(1) | 0.5185185185185185    | 0.272108843537415   | 0.6280487804878049   |
+---------------+-----------------------+---------------------+----------------------+
| IR_encoded(2) | 0.47883597883597884   | 0.717687074829932   | 0.36585365853658536  |
+---------------+-----------------------+---------------------+----------------------+

CPD of EI_encoded
+---------------+---------------+----

...


Learned Structure (Edges) for 9000 samples (AIC):
[('EI_encoded', 'IR_encoded'), ('SP_encoded', 'IR_encoded'), ('SP_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+---------------------+-----+---------------------+---------------------+
| EI_encoded    | EI_encoded(0)       | ... | EI_encoded(2)       | EI_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | ... | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(0) | 0.16732617297908423 | ... | 0.11569416498993963 | 0.082687338501292   |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(1) | 0.4652345958168457  | ... | 0.5503018108651911  | 0.31976744186046513 |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(2) | 0.367439231

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9500 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('IR_encoded', 'SP_encoded'), ('SP_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+-----------+
| IR_encoded(0) | 0.862842  |
+---------------+-----------+
| IR_encoded(1) | 0.0991579 |
+---------------+-----------+
| IR_encoded(2) | 0.038     |
+---------------+-----------+

CPD of EI_encoded
+---------------+---------------------+-----+----------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(2)        | IR_encoded(2)       |
+---------------+---------------------+-----+----------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | ... | SP_encoded(1)        | SP_encoded(2)       |
+---------------+---------------------+-----+----------------------+---------------------+
| EI_encoded(0) | 0.7809477756286267  | ... | 0.45161290322580644  | 0.6444444444444445  |
+---------------+---------------------+-----+-------------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+---------------------+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(0) | 0.2545334652159578  | 0.22393822393822393 | 0.30370562321845035 |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(1) | 0.5858885591823277  | 0.46782496782496785 | 0.5172324436382483  |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(2) | 0.15957797560171447 | 0.3082368082368082  | 0.17906193314330138 |
+---------------+---------------------+---------------------+---------------------+

CPD of EI_encoded
+---------------+---------------------+-----+------------------

# ------------------------------------------------------------------------------------------------------------

# Hypothesis Model 500, 1000, 1500, ..., 10000 Samples (dense) 1 hidden Layer, 10 Neurons Relu

In [49]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Define the Neural Network architecture
def create_nn_model(hidden_layers=1, nodes_per_layer=10):
    model = models.Sequential()

    # Input layer (2 input features: IR_encoded and EI_encoded)
    model.add(layers.InputLayer(input_shape=(2,)))

    # Hidden layers
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(nodes_per_layer, activation='relu', name=f"hidden_layer_{layer_num + 1}"))

    # Output layer (3 classes: decrease, stable, increase)
    model.add(layers.Dense(3, activation='softmax', name="output_layer"))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Loop through each sample size
for size in sample_sizes:
    # Load data for the current sample size (adjust the file paths if necessary)
    outcomes_file = f'outcomes_dense_{size}.csv'

    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)
    df['SP_encoded'] = df['SP'].map(sp_map)

    # Features (IR and EI) and labels (SP)
    X = df[['IR_encoded', 'EI_encoded']]
    y = df['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print(f"\nSample size: {size}")
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model
    nn_model = create_nn_model(hidden_layers=1, nodes_per_layer=10)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(X_val, y_val),
                           verbose=0)  # Set verbose=0 to avoid too much output

    # Evaluate on the validation set
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")

    # Evaluate on the test set
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the IR, EI, predicted SP, and the NN probabilities
    result_df = pd.DataFrame({
        'IR': df['IR'][:len(predicted_labels)],  # IR column from the original dataframe
        'EI': df['EI'][:len(predicted_labels)],  # EI column from the original dataframe
        'Predicted_SP': predicted_labels         # Predicted SP labels
    })

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df, probs_df.reset_index(drop=True)], axis=1)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 5 rows):")
    print(combined_df.head(15))

# After the loop is done, print this message
print("\nLooping through all sample sizes complete!")


Sample size: 500
Training Data: (350, 2) (350,)
Validation Data: (75, 2) (75,)
Test Data: (75, 2) (75,)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 500 samples: 0.3867
Test Accuracy for 500 samples: 0.3067
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

Predicted Results and Probabilities for 500 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high     good       stable       0.184599     0.413439       0.401963
1      low  average       stable       0.184599     0.413439       0.401963
2      low     good       stable       0.118898     0.441380       0.439722
3      low     poor       stable       0.184599     0.413439       0.401963
4      low  average       stable       0.184599     0.413439       0.401963
5     high     poor       stable       0.208342     0.421427       0.370231
6      low  average       stable       0.208342     0.421427       0.370231
7      low  average       stable       0.118898     0.441380       0.439722
8   medium     poor       stable       0.314706     0.360656       0.324638
9      low     poor     increase       0.153446    

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 1000 samples: 0.4933
Test Accuracy for 1000 samples: 0.4800
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

Predicted Results and Probabilities for 1000 samples (First 5 rows):
      IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0    low     good     increase       0.259113     0.246560       0.494327
1   high  average     decrease       0.413746     0.250169       0.336086
2   high     poor       stable       0.387327     0.440073       0.172600
3   high     good       stable       0.136974     0.740133       0.122893
4    low     good     decrease       0.413746     0.250169       0.336086
5   high     poor       stable       0.387327     0.440073       0.172600
6   high  average       stable       0.136974     0.740133       0.122893
7    low     good       stable       0.223644     0.525338       0.251019
8   high  average     increase       0.259113     0.246560       0.494327
9   high  average     increase       0.320594     0.303499       0.3

...

Validation Accuracy for 9000 samples: 0.5081
Test Accuracy for 9000 samples: 0.4741
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     poor       stable       0.303665     0.450745       0.245590
1   medium     good     decrease       0.394312     0.251016       0.354672
2      low     poor     decrease       0.523760     0.242248       0.233992
3   medium  average       stable       0.338132     0.511933       0.149935
4     high     poor       stable       0.237191     0.403132       0.359677
5      low     poor       stable       0.205977     0.541235       0.252787
6      low     poor       stable       0.338132     0.511933       0.149935
7      low     poor       stable       0.237191     0.403132       0.359677
8      low     poor       stable       0.323967     0.391830       0.284204
9      low     poor     decrease       0.394312

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 9500 samples: 0.4933
Test Accuracy for 9500 samples: 0.4947
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9500 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     good     increase       0.151972     0.420493       0.427535
1      low     good     decrease       0.453110     0.256497       0.290393
2     high  average       stable       0.255299     0.376072       0.368629
3     high  average       stable       0.041479     0.733495       0.225025
4      low     good       stable       0.255299     0.376072       0.368629
5     high  average     increase       0.151972     0.420493       0.427535
6     high  average     increase       0.231452     0.306137       0.462411
7     high     good       stable       0.272090     0.415344       0.312566
8   medium     poor     decrease       0.453110     0.256497       0.290393
9      low     poor     increase       0.151972

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 10000 samples: 0.4593
Test Accuracy for 10000 samples: 0.4580
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 10000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low     good       stable       0.363933     0.372413       0.263655
1      low  average       stable       0.363933     0.372413       0.263655
2      low     good     decrease       0.501489     0.182556       0.315955
3   medium     poor       stable       0.354805     0.491198       0.153997
4      low     good     decrease       0.501489     0.182556       0.315955
5      low     good       stable       0.363933     0.372413       0.263655
6   medium     poor       stable       0.278338     0.453451       0.268210
7      low     good       stable       0.354805     0.491198       0.153997
8      low  average       stable       0.346791     0.416422       0.236787
9      low  average       stable       0.363

# Hypothesis Model 500, 1000, 1500, ..., 10000 Samples (sparse) 1 hidden Layer, 10 Neurons Relu

In [50]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Define the Neural Network architecture
def create_nn_model(hidden_layers=1, nodes_per_layer=10):
    model = models.Sequential()

    # Input layer (2 input features: IR_encoded and EI_encoded)
    model.add(layers.InputLayer(input_shape=(2,)))

    # Hidden layers
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(nodes_per_layer, activation='relu', name=f"hidden_layer_{layer_num + 1}"))

    # Output layer (3 classes: decrease, stable, increase)
    model.add(layers.Dense(3, activation='softmax', name="output_layer"))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Loop through each sample size
for size in sample_sizes:
    # Load data for the current sample size (adjust the file paths for sparse data)
    outcomes_file = f'outcomes_sparse_{size}.csv'

    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)
    df['SP_encoded'] = df['SP'].map(sp_map)

    # Features (IR and EI) and labels (SP)
    X = df[['IR_encoded', 'EI_encoded']]
    y = df['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print(f"\nSample size: {size}")
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model
    nn_model = create_nn_model(hidden_layers=1, nodes_per_layer=10)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(X_val, y_val),
                           verbose=0)  # Set verbose=0 to avoid too much output

    # Evaluate on the validation set
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")

    # Evaluate on the test set
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the IR, EI, predicted SP, and the NN probabilities
    result_df = pd.DataFrame({
        'IR': df['IR'][:len(predicted_labels)],  # IR column from the original dataframe
        'EI': df['EI'][:len(predicted_labels)],  # EI column from the original dataframe
        'Predicted_SP': predicted_labels         # Predicted SP labels
    })

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df, probs_df.reset_index(drop=True)], axis=1)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 5 rows):")
    print(combined_df.head(15))

# After the loop is done, print this message
print("\nLooping through all sparse sample sizes complete!")


Sample size: 500
Training Data: (350, 2) (350,)
Validation Data: (75, 2) (75,)
Test Data: (75, 2) (75,)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 500 samples: 0.5733
Test Accuracy for 500 samples: 0.4800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

Predicted Results and Probabilities for 500 samples (First 5 rows):
      IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0    low  average     increase       0.143132     0.213182       0.643686
1    low     poor     increase       0.075034     0.122164       0.802802
2    low  average     increase       0.143132     0.213182       0.643686
3    low  average     increase       0.266842     0.273357       0.459801
4   high  average     increase       0.243131     0.309835       0.447034
5    low  average     increase       0.075034     0.122164       0.802802
6    low  average     increase       0.143132     0.213182       0.643686
7    low  average     increase       0.143132     0.213182       0.643686
8    low  average     increase       0.143132     0.213182       0.643686
9    low  average     increase       0.143132     0.213182       0.6436

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 1000 samples: 0.3933
Test Accuracy for 1000 samples: 0.5067
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Predicted Results and Probabilities for 1000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high     good     increase       0.298668     0.323521       0.377811
1   medium  average     decrease       0.582541     0.251349       0.166110
2   medium     good     increase       0.353570     0.257290       0.389139
3   medium     good     decrease       0.424754     0.334132       0.241114
4     high     poor     decrease       0.705424     0.172635       0.121940
5     high     poor     increase       0.353570     0.257290       0.389139
6   medium  average     increase       0.298668     0.323521       0.377811
7     high     poor     decrease       0.582541     0.251349       0.166110
8     high  average     increase       0.298668     0.323521       0.377811
9   medium     poor     increase       0.321165 

...

Validation Accuracy for 9000 samples: 0.5207
Test Accuracy for 9000 samples: 0.5370
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium  average     increase       0.134901     0.264175       0.600923
1     high     poor     decrease       0.480053     0.280387       0.239560
2     high  average     increase       0.332436     0.164710       0.502854
3     high     poor     decrease       0.480053     0.280387       0.239560
4     high     good     increase       0.243307     0.149258       0.607435
5     high  average       stable       0.199392     0.448697       0.351911
6     high  average       stable       0.199392     0.448697       0.351911
7     high     good     increase       0.134901     0.264175       0.600923
8   medium     good     decrease       0.480053     0.280387       0.239560
9     high     good       stable       0.301109

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 9500 samples: 0.5530
Test Accuracy for 9500 samples: 0.5614
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Predicted Results and Probabilities for 9500 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low  average       stable       0.082460     0.571710       0.345831
1   medium  average     decrease       0.509572     0.213250       0.277178
2      low  average     increase       0.394141     0.051339       0.554520
3      low     poor       stable       0.082460     0.571710       0.345831
4      low     poor     increase       0.428676     0.110906       0.460418
5   medium  average     increase       0.394141     0.051339       0.554520
6      low     poor     increase       0.394141     0.051339       0.554520
7     high     poor     decrease       0.509572     0.213250       0.277178
8      low  average     increase       0.394141     0.051339       0.554520
9      low  average     increase       0.428676

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 10000 samples: 0.5153
Test Accuracy for 10000 samples: 0.5373
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 10000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high  average     decrease       0.726915     0.159989       0.113096
1   medium     poor     increase       0.177329     0.220979       0.601692
2     high  average     decrease       0.524345     0.350878       0.124778
3   medium  average       stable       0.284289     0.381862       0.333849
4      low     good     increase       0.307857     0.320185       0.371959
5   medium  average     decrease       0.524345     0.350878       0.124778
6   medium     poor     increase       0.117047     0.293174       0.589779
7      low     poor     increase       0.177329     0.220979       0.601692
8      low     good     increase       0.177329     0.220979       0.601692
9     high     poor       stable       0.284

# ------------------------------------------------------------------------------------------------------------

# K-L Divergence NN Dense Data

## This is the entropy for dense data distribution with a sample size of 500 - 10000 and a NN with 1 hidden layer and 10 Neurons.

In [51]:
# Define the function to save K-L divergence to a file
def save_kl_divergence(sample_size, kl_div_value):
    file_name = 'kl_div_NN_1_10_dense.csv'

    # Append the K-L divergence for this sample size to the file
    with open(file_name, 'a') as f:
        f.write(f"{sample_size},{kl_div_value:.4f}\n")

# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)  # Loop from 500 to 10,000 in increments of 500

# Loop through each sample size
for sample_size in sample_sizes:
    # Load the ground truth probabilities for the current sample size
    ground_truth_probs_file = f'probabilities_dense_{sample_size}.csv'
    df_gt_probs = pd.read_csv(ground_truth_probs_file)

    # Extract the SP probabilities from the Ground Truth model (dense Bayesian network)
    # Placeholder to store K-L divergence values
    kl_divergences = []

    # Load the predictions from memory (already generated by the NN)
    outcomes_file = f'outcomes_dense_{sample_size}.csv'
    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR and EI
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)

    # Assuming predictions were saved in memory from the NN run
    predictions = nn_model.predict(df[['IR_encoded', 'EI_encoded']])

    # Loop through the test set predictions
    for i in range(len(df)):
        # Neural Network predicted probabilities for SP (decrease, stable, increase)
        predicted_probs = predictions[i]

        # Get the IR and EI values for the current sample
        ir_value = df.iloc[i]['IR_encoded']
        ei_value = df.iloc[i]['EI_encoded']

        # Map encoded values back to original labels
        ir_value = {0: 'low', 1: 'medium', 2: 'high'}[ir_value]
        ei_value = {0: 'poor', 1: 'average', 2: 'good'}[ei_value]

        # Get the corresponding ground truth probabilities for SP given IR and EI
        col_prefix = f'SP_given_IR_{ir_value}_EI_{ei_value}'
        ground_truth_probs = df_gt_probs.filter(like=col_prefix).values.flatten()

        # Ensure the probabilities are non-zero to avoid division by zero
        epsilon = 1e-10
        ground_truth_probs = np.clip(ground_truth_probs, epsilon, 1)

        # Compute K-L divergence (Neural Network vs Ground Truth)
        kl_div = entropy(predicted_probs, ground_truth_probs)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence over all samples for the current sample size
    average_kl_divergence = np.mean(kl_divergences)

    # Save the K-L divergence value to a CSV file
    save_kl_divergence(sample_size, average_kl_divergence)

    # Print confirmation
    print(f"Average K-L Divergence for {sample_size} samples: {average_kl_divergence:.4f}")

# Once all sample sizes are processed, the K-L divergences will be saved in 'kl_div_NN_1_10_dense.csv'
print("\nK-L divergence calculations complete.")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Average K-L Divergence for 500 samples: 0.6167
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Average K-L Divergence for 1000 samples: 0.3102
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 1500 samples: 0.2202
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 2000 samples: 0.4451
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Average K-L Divergence for 2500 samples: 0.4494
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 3000 samples: 0.4133
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 3500 samples: 0.2037
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Average K-L Divergence for 4000 samples: 0.2856
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 4500 samples: 0.2514
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 5000 samples: 0.6741
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 5500 samples: 0.2019
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 

# K-L Divergence NN Sparse Data

## This is the entropy for sparse data distribution with a sample size of 500 - 10000 and a NN with 1 hidden layer and 10 Neurons.

In [52]:
# Define the function to save K-L divergence to a file at the end
def save_kl_divergences_to_file(kl_divergence_data):
    file_name = 'K-L Divergence sparse NN 1_10.csv'  # Changed file name to indicate sparse data

    # Save the K-L divergences to a CSV file
    with open(file_name, 'w') as f:
        f.write('Size,NN_Sparse_1_10_Entropy\n')  # Updated the header to indicate sparse data
        for sample_size, kl_div_value in kl_divergence_data.items():
            f.write(f"{sample_size},{kl_div_value:.4f}\n")

# Placeholder to store K-L divergence values for all sample sizes
kl_divergence_results = {}

# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    # Load the ground truth probabilities for the current sample size (sparse data)
    ground_truth_probs_file = f'probabilities_sparse_{sample_size}.csv'  # Changed to sparse data file
    df_gt_probs = pd.read_csv(ground_truth_probs_file)

    # Load outcomes for the current sample size (sparse data)
    outcomes_file = f'outcomes_sparse_{sample_size}.csv'  # Changed to sparse data file
    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)

    # Placeholder to store K-L divergence values
    kl_divergences = []

    # Loop through the test set predictions
    for i in range(len(df)):
        # Neural Network predicted probabilities for SP (decrease, stable, increase)
        predicted_probs = predictions[i]  # Assuming the NN predictions are already available in memory for this sample size

        # Get the IR and EI values for the current sample
        ir_value = df.iloc[i]['IR_encoded']
        ei_value = df.iloc[i]['EI_encoded']

        # Map encoded values back to original labels
        ir_value = {0: 'low', 1: 'medium', 2: 'high'}[ir_value]
        ei_value = {0: 'poor', 1: 'average', 2: 'good'}[ei_value]

        # Get the corresponding ground truth probabilities for SP given IR and EI
        col_prefix = f'SP_given_IR_{ir_value}_EI_{ei_value}'
        ground_truth_probs = df_gt_probs.filter(like=col_prefix).values.flatten()

        # Ensure the probabilities are non-zero to avoid division by zero
        epsilon = 1e-10
        ground_truth_probs = np.clip(ground_truth_probs, epsilon, 1)

        # Compute K-L divergence (Neural Network vs Ground Truth)
        kl_div = entropy(predicted_probs, ground_truth_probs)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence over all samples for the current sample size
    average_kl_divergence = np.mean(kl_divergences)

    # Store the result for this sample size
    kl_divergence_results[sample_size] = average_kl_divergence

    # Print confirmation for each sample size
    print(f"Average K-L Divergence for {sample_size} sparse samples: {average_kl_divergence:.4f}")

# Once all sample sizes are processed, save the results to a CSV file
save_kl_divergences_to_file(kl_divergence_results)

# Print completion message
print("\nK-L divergence calculations complete and saved to 'K-L Divergence sparse NN 1_10.csv'.")

Average K-L Divergence for 500 sparse samples: 0.5892
Average K-L Divergence for 1000 sparse samples: 0.2951
Average K-L Divergence for 1500 sparse samples: 0.2273
Average K-L Divergence for 2000 sparse samples: 0.5658
Average K-L Divergence for 2500 sparse samples: 0.2875
Average K-L Divergence for 3000 sparse samples: 0.4919
Average K-L Divergence for 3500 sparse samples: 0.3117
Average K-L Divergence for 4000 sparse samples: 0.2632
Average K-L Divergence for 4500 sparse samples: 0.3158
Average K-L Divergence for 5000 sparse samples: 0.3611
Average K-L Divergence for 5500 sparse samples: 0.2915
Average K-L Divergence for 6000 sparse samples: 0.4458
Average K-L Divergence for 6500 sparse samples: 0.2278
Average K-L Divergence for 7000 sparse samples: 0.4178
Average K-L Divergence for 7500 sparse samples: 0.2448
Average K-L Divergence for 8000 sparse samples: 0.4186
Average K-L Divergence for 8500 sparse samples: 0.7007
Average K-L Divergence for 9000 sparse samples: 0.2301
Average K-L

# ------------------------------------------------------------------------------------------------------------